In [ ]:
# ==============================================================================
# プロジェクト: MLBにおけるバント戦略のゲーム理論的分析 (2024 Full Season)
# 作成日: 2026/02/10
# 概要: データ取得から前処理、利得行列計算、スプレーチャート作成までの完全版コード
# ==============================================================================

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pybaseball import statcast
import warnings

# グラフのスタイル設定 (ブログ用に見やすく)
sns.set_style("white")
warnings.filterwarnings('ignore')

# ------------------------------------------------------------------------------
# 1. データ取得 (Data Acquisition)
# ------------------------------------------------------------------------------
print("🚀 Step 1: 2024年フルシーズンデータを取得中... (数分かかります)")
# 期間: 2024年開幕(3/28)〜閉幕(10/1)
df_full = statcast(start_dt='2024-03-28', end_dt='2024-10-01')
print(f"✅ 取得完了: 全 {len(df_full)} 行")

# ------------------------------------------------------------------------------
# 2. 前処理 (Preprocessing)
# ------------------------------------------------------------------------------
print("Step 2: データのクリーニングとフィルタリング...")

# 欠損値埋め
df_full[['on_1b', 'on_2b', 'on_3b']] = df_full[['on_1b', 'on_2b', 'on_3b']].fillna(0)

# 条件: 無死一塁 (No Outs, Runner on 1st)
condition = (
    (df_full['outs_when_up'] == 0) &
    (df_full['on_1b'] > 0) &
    (df_full['on_2b'] == 0) &
    (df_full['on_3b'] == 0)
)
df_analysis = df_full[condition].copy()

# ------------------------------------------------------------------------------
# 3. 戦略・結果・守備の判定ロジック (Logic Definitions)
# ------------------------------------------------------------------------------
def identify_strategy(row):
    """攻撃側の戦略: バント(Bunt) か 強攻(Swing) か"""
    desc = str(row['description']).lower()
    evt = str(row['events']).lower()
    bb = str(row['bb_type']).lower()
    speed = row['launch_speed']
    
    # 明示的なバント
    if 'bunt' in desc or 'bunt' in evt or 'bunt' in bb: return 'Bunt'
    # バント失敗の推論 (打球速度が45マイル未満のゴロアウト)
    if pd.notna(speed) and (speed < 45) and ('ground' in evt or 'field_out' in evt): return 'Bunt'
    return 'Swing'

def classify_result(evt):
    """結果の詳細分類"""
    s = str(evt).lower()
    if 'sac' in s: return 'Success (Sacrifice)'
    if any(x in s for x in ['single', 'double', 'triple', 'home_run']): return 'Success (Hit)'
    if 'error' in s: return 'Success (Error)'
    if any(x in s for x in ['out', 'play', 'choice']): return 'Failure (Out)'
    return 'Other'

def identify_defense(val):
    """守備側の戦略: シフト(Shift) か 定位置(Normal) か"""
    val = str(val).lower()
    if 'strategic' in val or 'shift' in val: return 'Shift'
    return 'Normal'

# ロジック適用
df_analysis['strategy'] = df_analysis.apply(identify_strategy, axis=1)
df_analysis['result_type'] = df_analysis['events'].apply(classify_result)
df_analysis['defense'] = df_analysis['if_fielding_alignment'].apply(identify_defense)

# 数値型変換 (利得計算用)
df_analysis['delta_run_exp'] = pd.to_numeric(df_analysis['delta_run_exp'], errors='coerce')

print(f"✅ 分析対象抽出完了: {len(df_analysis)} 件 (無死一塁)")

# ------------------------------------------------------------------------------
# 4. 分析A: ゲーム理論・利得行列 (Payoff Matrix)
# ------------------------------------------------------------------------------
print("Step 3: 利得行列の計算と描画...")

payoff = df_analysis.groupby(['strategy', 'defense'])['delta_run_exp'].mean().unstack().astype(float)

plt.figure(figsize=(8, 6))
sns.heatmap(payoff, annot=True, fmt='.4f', cmap='RdBu', center=0, annot_kws={'size': 16})
plt.title('Payoff Matrix: Change in Run Expectancy (2024 Full Season)', fontsize=14)
plt.xlabel('Defense Strategy', fontsize=12)
plt.ylabel('Offense Strategy', fontsize=12)
plt.savefig('final_payoff_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("   -> 利得行列: final_payoff_matrix.png に保存しました")

# ------------------------------------------------------------------------------
# 5. 分析B: スプレーチャート作成 (Spray Chart)
# ------------------------------------------------------------------------------
print("Step 4: スプレーチャートの作成...")

# バント抽出 & 座標あり & 左右打席あり
df_bunt = df_analysis[
    (df_analysis['strategy'] == 'Bunt') &
    (df_analysis['hc_x'].notna()) & 
    (df_analysis['hc_y'].notna())
].copy()

# L/R欠損ケア
df_bunt['stand'] = df_bunt['stand'].fillna('Unknown')
df_bunt = df_bunt[df_bunt['stand'].isin(['L', 'R'])]

# 描画順序の制御 (青い点を一番上に表示するため)
df_bunt['z_order'] = df_bunt['result_type'].map({
    'Other': 0,
    'Success (Sacrifice)': 1,
    'Failure (Out)': 2,
    'Success (Error)': 3,
    'Success (Hit)': 4  # 最優先描画
})
df_bunt = df_bunt.sort_values('z_order')

# プロット作成
g = sns.FacetGrid(df_bunt, row="stand", col="defense", hue="result_type",
                  height=5, aspect=1,
                  palette={"Success (Sacrifice)": "green",
                           "Success (Hit)": "blue",
                           "Success (Error)": "cyan",
                           "Failure (Out)": "red",
                           "Other": "gray"},
                  hue_order=["Success (Sacrifice)", "Success (Hit)", "Success (Error)", "Failure (Out)", "Other"])

g.map(sns.scatterplot, "hc_x", "hc_y", s=100, alpha=0.9, edgecolor="white")
g.set(xlim=(0, 250), ylim=(250, 0)) # Y軸反転

# グラウンド描画関数
def draw_field(ax):
    bases_x, bases_y = [125, 175, 125, 75], [200, 150, 100, 150]
    ax.plot([125, 175], [200, 150], color='black', alpha=0.3)
    ax.plot([175, 125], [150, 100], color='black', alpha=0.3)
    ax.plot([125, 75], [100, 150], color='black', alpha=0.3)
    ax.plot([75, 125], [150, 200], color='black', alpha=0.3)
    ax.scatter(bases_x, bases_y, marker='D', color='gray', s=30)
    ax.axvline(125, color='gray', linestyle='--', alpha=0.2) # センターライン

for ax in g.axes.flat:
    draw_field(ax)

g.add_legend(title="Outcome")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Final Bunt Analysis: Location by Batter Hand & Defense', fontsize=16)

plt.savefig('final_bunt_spray_chart.png', dpi=300, bbox_inches='tight')
plt.show()

print("   -> スプレーチャート: final_bunt_spray_chart.png に保存しました")
print("🎉 全工程完了！お疲れ様でした！")